In [1]:
!pip install streamlit
!pip install pyngrok
!pip install tensorflow numpy pillow opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
import plotly.express as px
import time

# CSS tùy chỉnh cho giao diện
st.markdown("""
    <style>
    .main {
        background: linear-gradient(to right, #f0f4c3, #e1bee7);
        padding: 20px;
        border-radius: 10px;
    }
    .stButton>button {
        background-color: #4CAF50;
        color: white;
        border-radius: 5px;
        padding: 10px 20px;
        font-size: 16px;
    }
    .stFileUploader label {
        font-size: 16px;
        color: #2e7d32;
    }
    .title {
        font-size: 36px;
        color: #d81b60;
        text-align: center;
        font-weight: bold;
        margin-bottom: 10px;
    }
    .subtitle {
        font-size: 18px;
        color: #424242;
        text-align: center;
        margin-bottom: 20px;
    }
    .result-box {
        background-color: #ffffff;
        padding: 15px;
        border-radius: 10px;
        box-shadow: 0 4px 8px rgba(0,0,0,0.1);
        margin-top: 10px;
    }
    .expander-header {
        font-size: 18px;
        color: #d81b60;
    }
    </style>
""", unsafe_allow_html=True)

# Sidebar
st.sidebar.title("🌸 Thông tin mô hình")
st.sidebar.markdown("""
- **Mô hình**: VGG16 (Transfer Learning)
- **Dataset**: Flowers Recognition (~4242 ảnh)
- **Lớp**: Cúc, Bồ Công Anh, Hoa Hồng, Hướng Dương, Tulip
- **Độ chính xác**: >90% trên tập test
- **Hướng dẫn**:
  1. Tải ảnh hoa (jpg, png, jpeg).
  2. Chờ dự đoán và xem thông tin chi tiết.
  3. Nhấn "Xóa ảnh" để thử ảnh mới.
""")
st.sidebar.image("https://cdn.tgdd.vn/GameApp/3/227855/Screentshots/plantsnap-app-nhan-dien-cay-xac-dinh-loai-hoa-bang-tri-01-01-2022-11.png", width=100)
st.sidebar.markdown("---")
st.sidebar.caption("Ứng dụng bởi: Nhóm 19")

# Tiêu đề chính
st.markdown('<div class="title">Phân loại Hoa bằng VGG16 🌷</div>', unsafe_allow_html=True)
st.markdown('<div class="subtitle">Tải lên hình ảnh để khám phá loài hoa và thông tin chi tiết!</div>', unsafe_allow_html=True)

# Tên các loài hoa và thông tin chi tiết
class_names = ['Cúc', 'Bồ Công Anh', 'Hoa Hồng', 'Hướng Dương', 'Tulip']

flower_info = {
    'Cúc': {
        'Nguồn gốc': 'Châu Âu và châu Á',
        'Đặc điểm': 'Hoa nhỏ, màu trắng hoặc vàng, tâm vàng',
        'Kích thước': '20-30 cm',
        'Ý nghĩa': 'Trong sáng, thuần khiết'
    },
    'Bồ Công Anh': {
        'Nguồn gốc': 'Ôn đới châu Âu, Bắc Mỹ',
        'Đặc điểm': 'Hoa vàng, cụm tròn, hạt phát tán gió',
        'Kích thước': '10-30 cm',
        'Ý nghĩa': 'Tự do, hy vọng'
    },
    'Hoa Hồng': {
        'Nguồn gốc': 'Châu Á',
        'Đặc điểm': 'Cánh dày, màu đa dạng, có gai',
        'Kích thước': '50-200 cm',
        'Ý nghĩa': 'Tình yêu, lãng mạn'
    },
    'Hướng Dương': {
        'Nguồn gốc': 'Bắc Mỹ',
        'Đặc điểm': 'Hoa lớn, vàng tươi, tâm nâu',
        'Kích thước': '1-3 mét',
        'Ý nghĩa': 'Lạc quan, hướng sáng'
    },
    'Tulip': {
        'Nguồn gốc': 'Trung Đông, châu Âu',
        'Đặc điểm': 'Cánh tròn/oval, màu rực rỡ',
        'Kích thước': '10-70 cm',
        'Ý nghĩa': 'Thịnh vượng, tình yêu viên mãn'
    }
}

# Tải mô hình
@st.cache_resource
def load_model():
    with st.spinner("Đang tải mô hình VGG16..."):
        progress = st.progress(0)
        for i in range(100):
            time.sleep(0.02)
            progress.progress(i + 1)
        model = tf.keras.models.load_model('/VGG16_model.keras')
    return model

try:
    model = load_model()
    st.success("Mô hình VGG16 đã tải thành công! 🎉")
except Exception as e:
    st.error(f"Lỗi tải mô hình: {e}")
    st.stop()

# Chức năng tiền xử lý ảnh
def preprocess_image(image):
    img = cv2.resize(image, (224, 224))
    img = np.array(img).reshape(-1, 224, 224, 3)
    img = img / 255.0
    return img

# Khởi tạo trạng thái xóa file
if 'clear_file' not in st.session_state:
    st.session_state.clear_file = False

# Giao diện tải ảnh
st.markdown("---")
col1, col2 = st.columns([1, 2])

with col1:
    st.subheader("📸 Tải ảnh hoa")
    uploaded_file = st.file_uploader("Chọn ảnh (jpg, png, jpeg)", type=['jpg', 'png', 'jpeg'], key="uploader")

    if uploaded_file and not st.session_state.clear_file:
        # Hiển thị ảnh
        image = Image.open(uploaded_file)
        st.image(image, caption='Ảnh đã tải lên', use_container_width=True)


with col2:
    if uploaded_file and not st.session_state.clear_file:
        st.subheader("🌺 Kết quả phân loại")
        with st.spinner("Đang dự đoán..."):
            # Chuyển đổi và dự đoán
            image_np = np.array(image)
            img_array = preprocess_image(image_np)
            predictions = model.predict(img_array)

            # Lấy nhãn và độ tin cậy
            predicted_class_index = np.argmax(predictions, axis=1)[0]
            predicted_class_name = class_names[predicted_class_index]
            confidence = np.max(predictions[0]) * 100

            # Đặt xác suất < 0.001 thành 0
            adjusted_predictions = np.where(predictions[0] < 0.001, 0, predictions[0])

            # Lọc các lớp có xác suất > 0
            filtered_indices = [i for i, prob in enumerate(adjusted_predictions) if prob > 0]
            filtered_names = [class_names[i] for i in filtered_indices]
            filtered_values = [adjusted_predictions[i] * 100 for i in filtered_indices]

            # Hiển thị kết quả
            st.markdown(f'<div class="result-box"><b>Dự đoán</b>: {predicted_class_name}<br><b>Độ tin cậy</b>: {confidence:.2f}%</div>', unsafe_allow_html=True)

            # Thông tin chi tiết
            with st.expander("📖 Thông tin chi tiết", expanded=True):
                flower_details = flower_info.get(predicted_class_name, "Không có thông tin")
                for key, value in flower_details.items():
                    st.markdown(f"**{key}**: {value}")

            # Biểu đồ tròn xác suất (chỉ hiển thị lớp có xác suất > 0)
            st.markdown("📊 **Xác suất từng lớp**")
            if filtered_names:  # Kiểm tra xem có lớp nào để hiển thị không
                fig = px.pie(
                    names=filtered_names,
                    values=filtered_values,
                    color=filtered_names,
                    color_discrete_sequence=px.colors.qualitative.Pastel,
                    title="Phân bố xác suất dự đoán",
                    hole=0.3
                )
                fig.update_traces(textinfo='percent+label', pull=[0.1 if name == predicted_class_name else 0 for name in filtered_names])
                fig.update_layout(height=350, margin=dict(t=50, b=10, l=10, r=10))
                st.plotly_chart(fig, use_container_width=True)
            else:
                st.write("Không có lớp nào có xác suất lớn hơn 0.")

# Footer
st.markdown("---")
st.markdown("<p style='text-align: center; color: #757575;'>Ứng dụng phân loại hoa sử dụng VGG16 </p>", unsafe_allow_html=True)

Overwriting app.py


In [ ]:
from pyngrok import ngrok
!ngrok authtoken 2yM3XjfhLOpY90KqqVFWNJLVo8k_4d9w3BUmd1nrnHmsFgTRc
!streamlit run app.py &>/dev/null&
public_url = ngrok.connect(8501)
print(f"Truy cập ứng dụng tại: {public_url}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Truy cập ứng dụng tại: NgrokTunnel: "https://537f-35-222-3-80.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
from pyngrok import ngrok
tunnels = ngrok.get_tunnels()
print(tunnels)

[<NgrokTunnel: "https://d696-35-222-3-80.ngrok-free.app" -> "http://localhost:8501">, <NgrokTunnel: "https://adc8-35-222-3-80.ngrok-free.app" -> "http://localhost:8501">]


In [ ]:
from pyngrok import ngrok
ngrok.kill()  # Ngắt tất cả tunnel